# 使用 Numba 实现高斯模糊

让我们尝试做一些更复杂的事情。我们将使用 GPU 对图像应用高斯模糊。

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
from numba import cuda
import numpy as np
import math

plt.rcParams["figure.figsize"] = (30, 4)

## 数据加载

我们可以将图像文件读取为 NumPy 数组。让我们使用 Numba 的 logo。

In [ ]:
!wget https://raw.githubusercontent.com/NVIDIA/accelerated-computing-hub/main/gpu-python-tutorial/images/numba.png
im = plt.imread("numba.png")

In [ ]:
type(im)

In [ ]:
plt.imshow(im)

让我们将图像移动到 GPU，并创建一个输出数组用于存储我们的模糊图像。

In [ ]:
gpu_im = cuda.to_device(im)
gpu_output = cuda.to_device(np.zeros_like(gpu_im))

## 多维索引

在编写模糊函数之前，让我们讨论一下多维索引。在前面的示例中，我们使用了 `cuda.grid(1)` 来获取 `i` 值。这个调用中的 `1` 指的是这个索引应该具有的维度数。我们使用的是一维数组，所以使用一维索引是有意义的。

现在我们正在处理一个具有三个维度的图像：`x` 和 `y` 用于位置，以及 `channel`（红、绿、蓝、透明度）。

In [ ]:
im.shape

我们希望我们的 CUDA 内核在这个图像的每个像素上运行。我们可以继续使用 `i`，确保 `i` 是 `116 * 434 * 4`，并自己解开索引以确定我们想要的像素。

但是我们可以使用三维索引，这样我们就可以有 `x`、`y` 和 `channel` 索引，而不是 `i`。

首先，我们需要指定一个三维线程块大小。让我们继续使用线程块大小 `128`，但我们可以将其指定为三个相乘等于 `128` 的数字。

In [ ]:
# threadsperblock = 128
threadsperblock = (2, 16, 4)

接下来，我们需要计算我们的网格大小。我们将使用图像的维度来计算覆盖整个图像所需的线程数。

In [ ]:
blockspergrid_x = math.ceil(gpu_im.shape[0] / threadsperblock[0])
blockspergrid_y = math.ceil(gpu_im.shape[1] / threadsperblock[1])
blockspergrid_z = math.ceil(gpu_im.shape[2] / threadsperblock[2])
blockspergrid = (blockspergrid_x, blockspergrid_y, blockspergrid_z)

In [ ]:
blockspergrid

如果我们将线程和块相乘，我们可以看到我们的网格略大于我们的图像。

In [ ]:
[t * b for t, b in zip(threadsperblock, blockspergrid)]

## 编写我们的模糊内核

我们的内核需要接收图像和输出数组。

它需要获取要操作的网格位置，然后在该位置对图像执行模糊处理。

我们只关心在 `x` 和 `y` 维度上进行模糊处理，而不是在颜色通道上。所以我们只需直接传递 `c` 值。

In [ ]:
@cuda.jit
def blur(im, output):
    # 使用我们的三维网格，我们可以在三个维度上获取索引位置
    x, y, c = cuda.grid(3)

    # 因为我们的网格略大于图像，所以图像外的任何内容都应该被忽略
    if x < im.shape[0] and y < im.shape[1] and c < im.shape[2]:

        # 将输出数组像素设置为输入数组上该点周围九个像素的平均值
        output[x, y, c] = (im[x-1, y-1, c] + im[x, y-1, c] + im[x+1, y-1, c]  + \
                           im[x-1, y, c]   + im[x, y, c]   + im[x+1, y, c]    + \
                           im[x-1, y+1, c] + im[x+1, y, c] + im[x+1, y+1, c]) / 9

## 运行我们的内核

现在让我们多次运行我们的内核以获得所需的模糊程度。

我们需要每次传递在开始下一次之前完成。我们还需要我们的输出成为我们的输入，并且需要一个新的输出来工作。我们可以重用旧的输入作为新的输出数组。这很有效，因为我们可以重用 GPU 上的两个现有数组。

这就是手动内存管理派上用场的地方。我们可以多次调用我们的内核，但将所有数据留在 GPU 上。我们只需在每次传递之间交换指针。

In [ ]:
for i in range(5):
    blur[blockspergrid, threadsperblock](gpu_im, gpu_output)
    gpu_im, gpu_output = gpu_output, gpu_im

现在如果我们查看我们的图像，它应该足够模糊了。

In [ ]:
plt.imshow(gpu_output.copy_to_host())